In [59]:
# Import libraries
import numpy as np
import geopandas as gpd
import momepy
import networkx as nx
# import pandas as pd
# import shapely
# import shapely.geometry as sg
# import matplotlib
# import matplotlib.pyplot as plt
# %matplotlib inline

from lmzintgraf_gp_pref_elicit import dataset, gaussian_process, acquisition_function
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_ccs as utils_ccs
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_data as utils_data
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_experiment as utils_experiment
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_parameters as utils_parameters
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_user as utils_user

In [60]:
map = gpd.read_file("Sidewalk_width_crossings_small.geojson") #Read in the map with radius 250m and ~1000 nodes

# Objectives
objective1 = map['length']
objective2 = map['crossing']
objective3 = map['obstacle_free_width']

objectives = ('length', 'crossings')

In [61]:
# Create a NetworkX graph from the map
G = momepy.gdf_to_nx(map, approach='primal')
nodes = G.nodes
edges = G.edges

/usr/local/anaconda3/envs/test/lib/python3.10/site-packages/momepy/utils.py:252: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_network[length] = gdf_network.geometry.length


In [62]:
#Pick random ones or pick manually that make sense - to experiment
S = (120548.6120283842, 486088.19577846595)
T = (120798.0893320718, 486128.7633437495)

In [74]:
# Single-objective value iteration: v_n \gets {\arg\min}_{n' \in N_G(n)} c(n,n')+v_{n'}

def single_vi(graph, max_iter=1000, gamma=1.0, threshold=1e-8):
    # For smaller epsilon values, we need higher number of iterations for convergence,
    # but this way, we get more accurate results.

    num_nodes = len(graph)

    v_n = np.zeros(num_nodes) #Initialise each node to 0

    for i in range(max_iter): #or until convergence
        v_n_copy = np.copy(v_n)

        for n in graph:
            if not graph[n]: #We've reached terminal state
                continue
            max_value = -np.inf

            for action in graph[n]: #action=going from n to n′
                n_next = action
                cost = graph[n][action]
                result = cost + gamma * v_n_copy[n_next]
                max_value = max(max_value, result)

            v_n[n] = max_value

        if np.max(np.abs(v_n - v_n_copy)) < threshold: #Check for convergence
            break

    return v_n

In [76]:
# GG = nx.Graph()
# GG.add_nodes_from([1,2,3,4,5])
# GG.add_edges_from([(1,2), (1,3), (2,4), (4,5), (4,3), (3,5)])
# GG[1][2]['cost'] = 3
# GG[1][3]['cost'] = 1
# GG[2][4]['cost'] = 2
# GG[3][5]['cost'] = 7
# GG[4][3]['cost'] = 1
# GG[3][5]['cost'] = 4

single_graph = {
    0: {1: 3, 2: 1},
    1: {3: 2, 2: 1},
    2: {4: 4, 3: 1},
    3: {4: 7},
    4: {}
}
# Compute the optimal value vector
single_vi_results = single_vi(single_graph)
print(single_vi_results )
for state, value in enumerate(single_vi_results):
    print(f"V({state}) = {value}")

[12.  9.  8.  7.  0.]
V(0) = 12.0
V(1) = 9.0
V(2) = 8.0
V(3) = 7.0
V(4) = 0.0


In [65]:
# Depth-first search

In [69]:
# Multi-objective value iteration

# multi_graph = {
#     0: {1: [3,1], 2: [1,3]},
#     1: {3: [2,2], 2: [1,0]},
#     2: {4: [4,5], 3: [1,1]},
#     3: {4: [7,0]},
#     4: {}
# }
# # Compute the optimal value vector
# optimal_value = single_vi(multi_graph)
# print(optimal_value)
# for state, value in enumerate(optimal_value):
#     print(f"V({state}) = {value}")
